In [1]:
import os
import scvi
import scgen
import rpy2
import scib
import anndata
import logging
import warnings
import anndata2ri
import matplotlib
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sb
import scrublet as scr
import doubletdetection
from anndata import AnnData
from tabnanny import verbose
import matplotlib.pyplot as plt
from os import PathLike, fspath
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from matplotlib.pyplot import rcParams
from functions.deg_functions import deg_analyses
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from rpy2.robjects.conversion import localconverter

Global seed set to 0
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/phenograph/cluster.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` n

In [2]:
def get_sys_dpi(width, height, diag):
    '''
    obtain dpi of system
    
    w: width in pixels (if unsure, go vist `whatismyscreenresolution.net`)
    h: height in pixels
    d: diagonal in inches
    '''
    w_inches = (diag**2/ (1 + height**2/width**2))**0.5
    return round(width/w_inches)

In [3]:
# # Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# # Automatically convert rpy2 outputs to pandas dataframes
# pandas2ri.activate()
# anndata2ri.activate()
# %load_ext rpy2.ipython

warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

rcParams['figure.dpi'] = get_sys_dpi(1512, 982, 14.125)
#rcParams['figure.figsize']=(4,4) #rescale figures

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()



/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:356: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/rpy2/robjects/numpy2ri.py:245: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/rpy2/robjects/conversion.py:28: DeprecationWarning: The use of converter in module rpy2.robjects.conversion is deprecated. Use rpy2.robjects.conversion.get_conversion() instead of rpy2.robjects.conversion.converter.
  warnings.warn(

-----
anndata     0.8.0
scanpy      1.9.1
-----
OpenSSL                     22.0.0
PIL                         9.2.0
absl                        NA
adjustText                  NA
anndata2ri                  1.1
appnope                     0.1.2
asttokens                   NA
astunparse                  1.6.3
attr                        21.4.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
boto3                       1.26.32
botocore                    1.29.32
bottleneck                  1.3.5
brotli                      NA
certifi                     2022.09.24
cffi                        1.15.1
chex                        0.1.5
cloudpickle                 2.2.0
colorama                    0.4.4
contextlib2                 NA
cryptography                38.0.1
cycler                      0.10.0
cython_runtime              NA
dask                        2022.11.0
dateutil                    2.8.2
debugpy                     1.5.1
decorato

## Table of contents:

  * <a href=#Reading>1. Reading in the data</a>
  * <a href=#Preprocessing>2. Systematic differential analysis of gene expression</a>

# **1. Reading in the data**

### **Prepare data**

Now, we load the preprocessed and annotated data for downstream analysis.

In [4]:
save_prefix = 'mathys_pfc'

adata_annot = sc.read_h5ad(f'../data/processed/{save_prefix}/{save_prefix}_mapped_anndata.h5ad')
#adata_annot.obs['cell_type'] = adata_annot.obs['predictions'].copy()
#adata_annot = adata_annot[:, adata_annot.var.highly_variable=="True"]
adata_annot


AnnData object with n_obs × n_vars = 70310 × 16571
    obs: 'projid', 'tsne1', 'tsne2', 'pre.cluster', 'broad.cell.type', 'Subcluster', 'fastq', 'Subject', 'sample', 'libraryid', 'study', 'age_death', 'educ', 'msex_x', 'gpath_x', 'amyloid_x', 'plaq_n_x', 'cogdx_x', 'pathologic diagnosis of AD', 'amyloid_y', 'plaq_n_y', 'nft', 'tangles', 'cogn_global_lv', 'gpath_y', 'gpath_3neocort', 'amyloid.group', 'caa_4gp', 'ceradsc', 'braaksc', 'niareagansc', 'cogdx_y', 'msex_y', 'pathology.group', 'sampleid', 'cell_type', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'louvain_0.5', 'louvain_1.0', 'predictions'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Subject_colors', 'amyloid.group_colors', 'braaksc_colors', 'cell_type_colors', 'dendrogram_louvain

### **Prepare metadata**

Now we specify other related information

Specify the following:

- `metadata`: Path to metadata. Metadata must contain a column called `pathology.group` with the only unique groups being `no`, `early`, and `late`.

- `map_meta`: whether to map metadata to obtain `pathology.group`. If False, it will be assumed that `pathology.group` exist in `adata.obs`

- `reference_group`: Name of control group in metadata. This should ideally be `no`, representing the control group 

- `test_groups`: A list of the name of the group(s) in metadata/ `.obs` that should be treated as the test groups.d 

- `save_prefix`: Prefix for saving critical files. preferably chosen to be in the format `{source name}_{brain region}`. e.g `mathys_pfc`

- `filter_genes`: Specifies whether to filter genes using `gene_celltype_threshold` before before performing differential expression tests

- `grouped_test`: Specifies whether `early-` and `late-` pathology group should be grouped into `ad-pathology`. This should ideally be only set to `True` when `reference_group = no` and `test_groups = [late, early]`

        

In [5]:
map_meta = True
filter_genes = True
grouped_test = False
metadata = f'../data/raw/{save_prefix}/{save_prefix}_metadata.csv'
reference_group = 'early'                # name of the control group in metadata 
subject_id = 'Subject'                # for leng this is `PatientID` for mathys is 'Subject'
test_groups = ['late']      
gene_celltype_threshold = 0.05        # determines number of cells the gene must be expressed in

## 2.4 Systematic differential analysis of gene expression

[**Hansruedi Mathys et. al.**](https://doi.org/10.1038/s41586-019-1195-2) compared gene expression levels between `AD-pathology and no-pathology individuals in a cell type manner. The differential expression analysis was assessed using two tests. 

- **First**, a cell-level analysis was performed using the Wilcoxon rank-sum test and FDR multiple-testing correction (`FDR-adjusted p-values`). 

- **Second**, a Poisson mixed model accounting for the individual of origin for nuclei and for unwanted sources of variability was performed using the R packages `lme4` and `RUV-seq`, respectively.


Next, we use the ` Wilcoxon rank-sum test` in `scapany.tl.rank_genes_group` comparing `AD-pathology` group to `no-pathology` such that the log foldchange is ;

$$ Log_{2} ({Mean\ Gene\ Expression\ in\ AD\ category\ of\ Cell\ Type\ x \over Mean\ Gene\ Expression\ in\ Normal\ category\ of\ Cell\ Type\ x})$$


##### Group Cells (Pathology / No-Pathology)

In [6]:
adata_annot = deg_analyses.prep_anndata(adata_annot, map_meta, metadata, 
                                        reference_group, test_groups, subject_id,
                                        grouped_test)

##### Cell-type Differential Expression with Wilcoxon Rank-Sum Test


To perform the differential expression between conditions within cell-clusters, we subset the full data set into cluster-specific data sets. These cluster subsets are filtered to only include genes that are expressed in the data to reduce the multiple testing burdeN. Importantly, we normalize and log transform the count matrix (stored in `adata.layers`) in a cell-type-specific manner, since each cell-group is tested independently. Finally we define the model we are testing, run the test, and postprocess the results.

In [7]:
adata_sub = deg_analyses.wilcoxon_de(adata_annot, reference_group, norm_method = 'actionet', filter=True,
                                    filter_by = 'prop', filter_thresh = 0.05, test_layer=None, grouped_test=grouped_test)


filtered out 7108 genes that are detected in less than 1739.6000000000001 cells
evaluating differential expression in Excitatory...
ranking genes
    finished: added to `.uns['wilcoxon_test_pathology']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:13)

filtered out 8497 genes that are detected in less than 455.35 cells
evaluating differential expression in Inhibitory...
ranking genes
    finished: added to `.uns['wilcoxon_test_pathology']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to 

Reported DEGs

Next, we load the reported DEGs from [**Hansruedi Mathys et. al.**](https://doi.org/10.1038/s41586-019-1195-2) for comparison in a later step

In [8]:
mathys_degs = {}
for key in adata_sub.keys():
    try:
        mathys_degs[key] = pd.read_excel(f'../data/raw/mathys_pfc_from_paper/degs/ad_vs_no/{key.lower()}_degs.xlsx')
        mathys_degs[key].rename({'Unnamed: 0': 'names'}, inplace=True)
        mathys_degs[key]['IndModel.absFC'] = np.abs(mathys_degs[key]['IndModel.FC'])

    except FileNotFoundError:
        continue

T-test

Next, we package the test results into a presentable format, annotating genes as `up-` or `down-` regulated, and ranking genes according to `adjusted p-values`.

In [9]:
degs_t_test = deg_analyses.get_degs(adata_sub, grouped_test, reference_group, test_groups, save_prefix)

#### **OPTIONAL: Using custom script**

In [ ]:
# import numpy as np
# import pandas as pd
# import scanpy as sc
# from scipy.stats import wilcoxon
# from statsmodels.stats.multitest import fdrcorrection

# def scRNA_diff_run_wilcox_test(adata, group_key):
#     group = adata.obs[group_key]
#     # remove genes that aren't expressed in at least 10 cells
#     sc.pp.filter_genes(adata, min_cells=10)
#     # Library size normalization
#     sc.pp.normalize_total(adata, target_sum=1e4)
#     # Perform Wilcoxon rank-sum test and get p-values
#     pvals = np.apply_along_axis(lambda x: wilcoxon(x[group == np.unique(group)[0]],
#                                                     x[group == np.unique(group)[1]]).pvalue,
#                                 axis=1, arr=adata.X)
#     # multiple testing correction
#     pvals_adj = fdrcorrection(pvals, alpha=0.05)[1]
#     # Compute mean expression values for each group
#     group_means = adata[:, group_key].X.mean(axis=1)
#     # Compute fold change
#     fc = np.log2(group_means[:, 0] + 1e-8) - np.log2(group_means[:, 1] + 1e-8)
#     # Create a DataFrame with gene names, adjusted p-values, mean expression values and fold change
#     result_df = pd.DataFrame({'gene.name': adata.var_names,
#                               'adj.pvals': pvals_adj,
#                               'temp.mean': group_means.ravel(),
#                               'FC': fc})
#     # Order the DataFrame by adjusted p-values
#     result_df = result_df.sort_values('adj.pvals')
#     return result_df


In [ ]:
# degs_t_test2 = {}
# for cell_type in adata_annot.obs.cell_type.unique():
#     adata = adata_annot[adata_annot.obs.cell_type == cell_type].copy()
#     adata.X = adata.layers['counts'].toarray() 
#     degs_t_test2[cell_type] = scRNA_diff_run_wilcox_test(adata, 'disease_group')

The authors assessed the consistency of DEGs detected using the cell-level analysis model (`Wilcoxon rank-sum test`) with those obtained with the Poisson mixed model (`lme4` combined with `RUV-seq`) by comparing the directionality and rank of DEGs in the two models. Considering the technical challenges, of using these packages, we instead benchmark the `rank-sum test` results against `MAST`, a generalized linear model for modelling scRNA-seq data in R, developed by [**Finak, G. et. al. 2015**](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-015-0844-5). In addition to accounting for the the sample (`Subject`) of origin for each nuclei, we include the cellular detection rate, the fraction of genes expressed in a cell as a sources of unwanted variation/variability. Sveral benchmarking studies have shown that MAST is comparitvely better than other methods for scRNA-seq differential expression testing and is a commonly used tool in scRNA-seq analysis. Thus, We perform differential testing using the `MAST package`. 

Since MAST is only available in R, we convert our AnnData object into an R object via `anndata2ri`. MAST requires its own data input format, `SingleCellAssy` instead of the `SingleCellExperiment` object produced by the `anndata2ri` conversion. So, to run MAST we thus first put the data into the SingleCellExperiment format, then convert the SCE object into MAST's expected SingleCellAssay (sca) object. 

#### MAST implementation details

Here, we perform differential testing for AD-pathology vs No-pathology `disease_group`s in each of the cell clusters. Since `MAST` incorporates a zero-inflated negative binomial model tests for differential expression using a hurdle model, it can be very computationally intensive for large data sizes. To remedy this, `we randomly split each data in a stratified manner to inlude <5000 cells across both conditions` while retaining the relative proportion of cells in both test groups (AD-pathology vs No-pathology). This is expected to retain the expression patterns in both tests groups, while speeding up computation

In the generalized linear mixed model (specified and fit with the `zlm()` function), we include the test covariate `disease_group` (AD-pathology vs No-pathology), the `Subject` ID (ROS1 -- ROS48), and the number of genes in the cells, reculated from the log normalized values stored in `adata.layers['log']`. The `Subject` IDs are included as random effects to account for unwanted variability/effects due to individual nuclei orgin, mouse-specific that may confound our results. The number of genes is added to fit the technical variability.

To test for differences over the `disease_group` covariate we perform a likelihood ratio test (in the `summary()` function call after fitting the model). 

In post-processing we correct for multiple testing using a Benjamini-Hochberg FDR correction (function `p.adjust()`) and map the Ensembl Gene IDs to gene symbols which are easier to read and interpret.

We provide python implementation of MAST which can run inline in the `mixed_model_DEG_analysis.ipynb` notebook  